In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

In [ ]:
## Parameters
CAL_INPUT_IMAGE_DIR = "camera_cal"
CAL_OUTPUT_FILE = "calibration_params.p"
IMG_SIZE = (960, 1280)
COLOR_THRESHOLD = (170, 255)
GRADIENT_X_THRESHOLD = (20, 100)
GRADIENT_MAG_THRESHOLD = (20, 100)
GRADIENT_DIR_THRESHOLD =(0.7, 1.3)

In [ ]:
# Step 1: Calibrate camera, if not done before
class Utils:
    
    @staticmethod
    def calibrate_camera(image_dir, nx, ny, image_size, output_file):
        '''
        Compute and return camera calibration parameters
        '''
        
        if os.path.isfile(CAL_OUTPUT_FILE):
            print("Calibration has been done before, will skip!")
            cal_params = pickle.load( open( CAL_OUTPUT_FILE, "rb" ) )
            return cal_params
        else:    
            objp = np.zeros((ny*nx, 3), np.float32)
            objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)
            objpoints = []
            imgpoints = []

            images = glob.glob(os.path.join(image_dir, '*.jpg'))
            for ids, fname in enumerate(images):
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
            if ret is True:
                objpoints.append(objp)
                imgpoints.append(corners)

            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
            calibration_params = {}
            calibration_params["mtx"] = mtx
            calibration_params["dist"] = dist
            f = open (CAL_OUTPUT_FILE, "wb")
            pickle.dump(calibration_params, f)
            print("Camera calibration parameters written to {}".format(CAL_OUTPUT_FILE))
            return calibration_params

    @statismethod
    def to_gray_scale(image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    @staticmethod
    def gaussian_blur(img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    
    @staticmethod
    def canny(img, low_threshold, high_threshold):
        """Applies the Canny transform"""
        return cv2.Canny(img, low_threshold, high_threshold)

    @staticmethod
    def gradient_x_threshold(gray_image, min_max):
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        abs_sobelx = np.absolute(sobelx)
        scaled_sobelx = np.uint8(255 * (abs_sobelx/(np.max(abs_sobelx))))
        binary = np.zeros_like(scaled_sobelx)
        binary[(scaled_sobelx > min_max[0]) & (scaled_sobelx <= min_max[1])] = 1
        return binary
    
    @staticmethod
    def color_threshold(image, min_max):
        hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
        channel_s = hls[:,:,2]
        binary = np.zeros_like(channel_s)
        binary[(channel_s > min_max[0]) & (channel_s <=min_max[1])] = 1
        return binary
    
    @staticmethod
    def grad_dir_threshold(gray_image, min_max):
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        angle = cv2.arctan(np.absolute(sobely), np.absolute(sobelx)
        binary =  np.zeros_like(angle)
        binary[(angle >= min_max[0]) & (angle <= min_max[1])] = 1
        return binary 

    @staticmethod                       
    def get_perspective_src_dst_points(sample_image):
        src_vertices = np.array([[(0.15 * x_size,y_size),(0.4 * x_size, 0.6 * y_size), (0.6 * x_size, 0.6 * y_size), (0.85 * x_size, y_size)]], dtype=np.int32)
        dst_vertices = np.array([[(0.15 * x_size,y_size),(0.15 * x_size, 0.6 * y_size), (0.85 * x_size, 0.6 * y_size), (0.85 * x_size, y_size)]], dtype=np.int32)
        return (src_vertices, dst_vertices)
                        
    @staticmethod                       
    def histogram(image):
        return np.sum(img[img.shape[0]/2:,:], axis=0)

            


In [ ]:
class Pipeline:
    
    def __init__():
        self.cal_mtx = None
        self.cal_dist = None
        self.pers_M = None
        self.left_fit = None
        self.right_fit = None
            
    def undistort(self, image):
        if self.cal_mtx is None:
            cal_params = Utils.calibrate_camera(CAL_INPUT_IMAGE_DIR, nx, ny)
            self.cal_mtx = cal_params["mtx"]
            self.cal_dist = cal_params["dist"]
        
        return cv2.undistort(image, mtx, dist, None, mtx)
        
    def apply_thresholds(self, image):
        gray = Utils.to_gray_scale(image)
        color_threshold_img    = Utils.color_threshold(image, COLOR_THRESHOLD)
        gradx_binary = Utils.gradient_x_threshold(gray, GRADIENT_X_THRESHOLD)
        mag_binary   = Utils.grad_mag_threshold(gray, GRADIENT_MAG_THRESHOLD)
        dir_binary   = Utils.grad_dir_threshold(gray, GRAD_DIR_THRESHOLD)
        combined = np.zeros_like(color_threshold_img)
        combined[(gradx_binary == 1) | ((mag_binary == 1) & (dir_binary == 1))] = 1
        return combined
    
    def warp_perspective(self, image_binary):
        if self.pers_M is None:
            src, dst = Utils.get_perspective_src_dst_points(image)
            self.pers_M = cv2.getPerspectiveTransform(src, dst)
        
        warped_binary = cv2.warpPerspective(image_binary, self.pers_M, image.shape, flags=cv2.INTER_LINEAR)
        return warped
    
    def find_lanes(self, warped_binary):
        if self.left_fit is None or self.right_fit is None:
            self.init_lane_polynomials(warped_binary)
        else:
            self.find_lane_polynomials(warped_binary):
                
            
    def init_lane_polynomials(self, warped_binary):
        # build histogram
        histogram = Utils.histogram(warped_binary)
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[0:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        n_windows = 9
        window_height = np.int(warped_binary.shape[0]/n_windows)
        non_zero = binary_warped.nonzero()
        non_zero_x = np.array(non_zero[0])
        non_zero_y = np.array(non_zero[1])
        
        left_x_current = leftx_base
        right_x_current = rightx_base
        margin = 100
        minpix = 50
        
        left_lane_inds = []
        right_lane_inds = []
        
        y_max = image.shape[0]
        x_max = image.shape[1]
        for window in range(n_windows):
            win_y_high = y_max - windows * window_height
            win_y_low  = win_y_high - window_height
            
            win_xleft_low  = left_x_current - margin
            win_xleft_high = left_x_current + margin
            win_xright_low  = right_x_current - margin
            win_xright_high = right_x_current + margin
            
            cv2.rectangle(out_img, (win_xleft_low, win_y_low),  (win_xleft_high, win_y_high), (0, 255, 0), 2)
            cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high), (0, 255, 0), 2)
            
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                left_x_current = np.int(np.mean(nonzero_x[good_left_inds]))
            if len(good_right_inds) > minpix:
                right_x_current = np.int(np.mean(nonzero_x[good_right_inds]))
                
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)

        
    def find_lane_polynomials(self, warped_binary):
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        
        # Fit a second order polynomial to each
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)
        
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
    def process(self, image):
        undistorted = self.undistort(image)
        binary_thresholded = self.apply_thresholds(undistorted)
        warped = self.warp_perspective(binary_thresholded)
        find_lanes(warped)
        return self.left_fit, self.right_fit

